In [ ]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_create_report
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = ''
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ""

outdir = setup("20220628_qc_from_june_briefing.ipynb")

In [ ]:
import pandas as pd
obs_fpath = "covid_model/analysis/20220628_july_gov_briefing/CDPHE Variant Data (manually transcribed and limited to model variants).csv"
obs = pd.read_csv(obs_fpath, parse_dates=['Date']).fillna(0)
obs['Date'] = [d.date() for d in obs['Date']]
obs = obs.set_index('Date')

In [ ]:
model_args = {
    #'base_spec_id': 3014,
    'params_defs': 'covid_model/analysis/20220628_july_gov_briefing/scenario_params.json'),
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/input/vacc_proj_params.json',
    'regions': ['co'],
    'mobility_mode': None,
    'start_date': '2020-01-24',
    'end_date': '2022-09-18',
    'max_step_size': 1.0,
    'ode_method': 'RK45'
}
# compute vacc effectiveness
frac_lt5 = 5.7/(5.7+6+6.4+6.3)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for hosp_frac in [0.3, 0.5, 0.8]:
    for school_contact_increase in [1.0, 1.1, 1.2]: 
        hrf = {"2020-01-01": 1, "2022-03-01": 0.8, "2022-03-15": 0.66, "2022-03-30": 0.5}
        school_contact = [{"param": "betta", "attrs": {'age': '0-19'}, "vals": {,"desc": ""},]
        lt5_vacc_adjsut = [{"param": "immunity",    "attrs": {'age': '0-19'}, "mults": {"2020-01-01": , "2021-12-23": 2, "2022-03-05": 300, "2022-03-9": 400, "2022-03-20": 0},"desc": ""},]
        scenario_args_list.append({'params_defs': base_model_args['params_defs'] + school_contact + ,
                                   'hosp_reporting_frac': hosp_frac,
                                   'tags': {}})
        for (weak_escape, strong_escape) in [(0.75, 0.1), (0.8, 0.2)]:

    # run the scenarios
    models = do_fit_scenarios(base_model_args, scenario_args_list, fit_args, multiprocess=multiprocess)

# create initial solution so we don't have to fit the whole window
model = CovidModel(**model_args)
model.prep(outdir=outdir)
model.solve_seir()
# now fit, but only the period of time which is different from the base spec id
fit_args = {
    #'fit_start_date': '2021-11-01',
    #'fit_end_date': '2022-05-01',
    'tc_min': 0.0,
    'tc_max': 0.999,
    'tc_window_size': 14,
    'tc_window_batch_size': 6,
    'tc_batch_increment': 2,
    'last_tc_window_min_size': 14,
    'outdir': outdir
}
model = do_single_fit(**fit_args, base_model = model, prep_model=False)

In [ ]:
model.solve_seir()
soln = model.solution_sum_df(['seir', 'variant'])